In [8]:
#! pip install ultralytics 
#! pip install psycopg2
import numpy as np
#import cv2
#import matplotlib.pyplot as plt
import os
import psycopg2
#import im_utils
import db_ops
import pandas as pd
import im_utils

import cv2

import tensorflow as tf
print(tf.__version__)

from keras.models import Model
from keras.saving import load_model

from ultralytics import YOLO

2.16.2


In [2]:

def bb_iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [3]:
test_lbls = ['5-hour energy', '99', 'AW', 'Acadia', 'Arizona Tea', 'Bang', 'Big Gulp', 'Bill Miller', 'Boulevard Brewing', 'Brisk', 'Bud', 'Budweiser', 'Burger King', 'Busch', 'Camel', 'Cheetos', 'Coca Cola', 'Coors', 'Corona', 'Dasani', 'Dasy', 'David', 'Deer Park', 'Dorritos', 'Dunkin Donuts', 'Evan Williams', 'Fanta', 'Fireball', 'Flow', 'Fruit Punch', 'Gatorade', 'GoGo', 'Google', 'Goya', 'Grace', 'Green Mountain', 'Heineken', 'Hersheys', 'Hill Country Fair', 'Icehouse Edge', 'Jarritos', 'Jim Beam', 'Kirkland', 'Kit Kat', 'Kool-Aid', 'La Yogurt', 'LaCroix', 'Lays', 'Lybbis', 'Marlboro', 'McDonalds', 'Mentos', 'Michelob', 'Miller', 'Milwakee Best Ice', 'MnM', 'Modelo', 'Monster', 'Mountain Dew', 'Natural Ice', 'New Amsterdam', 'Niagara', 'Oreo', 'Pabst Blue Ribbon', 'Pacifico', 'Pampers', 'Parliament', 'Peaceful Fruits', 'Perrier', 'Poland Spring', 'Popeyes', 'PurAqua', 'Redbull', 'Ross Ouzo', 'Sams Club', 'Samuel Adams', 'Sangria Senorial', 'Sanpple', 'Seven eleven', 'Shasta', 'Silver Falls', 'Slurpee', 'Smirnoff', 'Snickers', 'Sprite', 'Stella Artois', 'Turkey Hill', 'Twix', 'Two Ball', 'Vitamin', 'Welchs', 'Yoohoo']

In [4]:
label_path = '../random_test_dataset/train/labels'
files = os.listdir(label_path)
file_names = []
labels = []
lstX1 = []
lstY1 = []
lstX2 = []
lstY2 = []

for file in files:
    file_name = file.split('_jpg')[0]
    with open(label_path + '/'+file, 'r') as label_file:
        for line in label_file:
            idx, x1, y1, x2, y2 = line.split(' ')
            lstX1.append(float(x1))
            lstX2.append(float(x2))
            lstY1.append(float(y1))
            lstY2.append(float(y2))
            labels.append(test_lbls[int(idx)].lower())
            file_names.append(file_name)
        
test_labels_df = pd.DataFrame({'file_name':file_names,
                               'label': labels,
                                'x1': lstX1,
                                'x2': lstX2,
                                'y1': lstY1,
                                'y2': lstY2,}, index = range(len(file_names)))
test_labels_df

,file_name,label,x1,x2,y1,y2
0,IMG_1189,kirkland,0.453869,0.124008,0.369544,0.076885
1,IMG_2834,bud,0.496858,0.124008,0.294519,0.135169
2,IMG_2834,bud,0.748181,0.170304,0.608259,0.125248
3,IMG_4260,kirkland,0.559276,0.054563,0.426587,0.128968
4,IMG_8921,pabst blue ribbon,0.447715,0.073853,0.504162,0.050429
...,...,...,...,...,...,...
305,gMyVYZ2G2PM9Z8QlQc4Wt2QmwJI1bXHIZd6zEIIO,heineken,0.568321,0.050040,0.788399,0.068084
306,gMyVYZ2G2PM9Z8QlQc4Wt2QmwJI1bXHIZd6zEIIO,heineken,0.560150,0.047999,0.875543,0.068084
307,gMyVYZ2G2PM9Z8QlQc4Wt2QmwJI1bXHIZd6zEIIO,heineken,0.536664,0.035744,0.958607,0.081699
308,gMyVYZ2G2PM9Z8QlQc4Wt2QmwJI1bXHIZd6zEIIO,gatorade,0.699550,0.069445,0.296842,0.092594


In [16]:
image_path = '../random_test_dataset/train/images'
files = os.listdir(image_path)
image_names = [file.split('_jpg')[0] for file in files]

In [5]:
# load image classification model
classifier = load_model("./models/B0_EffNet_classifier.keras")

# load dataframe that correlated model index and database ID
id_idx_df = pd.read_csv('brand_ids_idx.csv', sep = '\t')

# load logo detection model
detector = YOLO('./models/logo_detector.pt')

In [17]:
os.path.join(image_path, files[0])

'../random_test_dataset/train/images/nUMK0KFKt1cBXBokYIliij3sXUKAOXcMr517O6mr_jpg.rf.de0c34f0f27673b2ad56d9936e63704e.jpg'

In [53]:
# Detection stage
boxes_df = pd.DataFrame(columns = ['im_name', 'xmin_coord', 'ymin_coord', 'xmax_coord', 'ymax_coord', 'box_conf', 'brand', 'brand_predicted'])

for file in files[0:3]:
    print('-'*30)
    try:
        # Scale the image and write to the local folder  
        image_original = cv2.imread(os.path.join(image_path, file))
        image_scaled = im_utils.ScaleImage(image_original, width = 640)
        
        # Run logo detector
        results = detector(image_scaled, conf = 0.25)
        
        # Find the saceling factor
        w_ratio = image_original.shape[1]/image_scaled.shape[1]
        h_ratio = image_original.shape[0]/image_scaled.shape[0]
        for i in range(len(results[0].boxes.xyxy)):   # Iterate over each box
            # Get the coordinates of the box on the original image
            xmin, ymin, xmax, ymax = map(int, results[0].boxes.xyxy[i]) 
            xmin = int(xmin * w_ratio)
            xmax = int(xmax * w_ratio)
            ymin = int(ymin * h_ratio)
            ymax = int(ymax * h_ratio)

            # Get the section of the original image that is in the box
            box_im = image_original[ymin:ymax, xmin:xmax,:]

            # Run the logo classifier on this box
            box_im = cv2.resize(box_im, (254,254))
            y_pred = classifier.predict(box_im.reshape(1,254,254,3))
            y_pred_dense = np.argmax(y_pred, axis=1)
            brand_name_df = id_idx_df[id_idx_df['idx'] == y_pred_dense[0]]
            
            # Save the box locally as an individual file
            idx = len(boxes_df)
            boxes_df.loc[idx, 'im_name'] = file.split('_jpg')[0]
            boxes_df.loc[idx, 'xmin_coord'] = xmin
            boxes_df.loc[idx, 'ymin_coord'] = ymin
            boxes_df.loc[idx, 'xmax_coord'] = xmax
            boxes_df.loc[idx, 'ymax_coord'] = ymax
            boxes_df.loc[idx, 'box_conf'] = float(results[0].boxes.conf[i])
            boxes_df.loc[idx, 'brand_conf'] = max(y_pred[0])
            boxes_df.loc[idx, 'brand_predicted'] = brand_name_df['brand_name'].iloc[0]
    except Exception as e:
        print(e)
boxes_df.to_csv(sep='\t')
    

------------------------------



0: 480x640 1 logo, 2856.2ms
Speed: 3.1ms preprocess, 2856.2ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 542ms/step
------------------------------

0: 480x640 (no detections), 2479.1ms
Speed: 2.0ms preprocess, 2479.1ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
------------------------------

0: 480x640 1 logo, 2615.8ms
Speed: 1.7ms preprocess, 2615.8ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 627ms/step


,im_name,xmin_coord,ymin_coord,xmax_coord,ymax_coord,box_conf,brand,brand_conf
0,nUMK0KFKt1cBXBokYIliij3sXUKAOXcMr517O6mr,2223,1183,2468,1361,0.643758,cheetos,0.998866
1,f1Nzx8QTyVH0ptKN6B3my3kq9jCESpgdpSdp1ShI,2419,1234,2520,1417,0.445805,zz_unbranded,0.96739


In [55]:
test_labels_df.head(3)

,file_name,label,x1,x2,y1,y2
0,IMG_1189,kirkland,0.453869,0.124008,0.369544,0.076885
1,IMG_2834,bud,0.496858,0.124008,0.294519,0.135169
2,IMG_2834,bud,0.748181,0.170304,0.608259,0.125248
